In [27]:
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

from scipy.io import arff

data = arff.loadarff('datasets/student_data.arff')[0]
X = np.empty((len(data), len(data[0]) - 1))
Y = np.empty((len(data), 1))


for i in range(len(data)):
# for entry in data:
    l: list = list(data[i])

    print('X está recebendo: ', l[:-1])
    print('Y está recebendo: ', l[-1])

    X[i] = l[:-1]
    Y[i] = l[-1]
    # np.append(X, l[:-1])
    # np.append(Y, l[-1])

x_tr, x_te, y_tr, y_te = train_test_split(X, Y, test_size = 0.2, random_state=42, stratify=Y)

print(X)
# print(x_te)
# print(len(y_tr))
# print(len(y_te))

dtc = DecisionTreeClassifier()
dtc.fit(x_tr, y_tr)

y_pred = dtc.predict(x_te)
accuracy_score(y_te, y_pred)

X está recebendo:  [b'F', b'G', b'Good', b'Good', b'Vg', b'Good', b'Y', b'Unmarried', b'V', b'Paid', b'Medium', b'Average', b'Um', b'10', b'Farmer', b'Housewife', b'Large', b'Poor', b'Govt', b'Asm', b'Small']
Y está recebendo:  b'Good'
X está recebendo:  [b'M', b'OBC', b'Vg', b'Vg', b'Vg', b'Vg', b'N', b'Unmarried', b'V', b'Paid', b'Low', b'Average', b'Um', b'Il', b'Service', b'Service', b'Small', b'Poor', b'Govt', b'Asm', b'Average']
Y está recebendo:  b'Average'
X está recebendo:  [b'F', b'OBC', b'Good', b'Good', b'Vg', b'Good', b'N', b'Unmarried', b'V', b'Paid', b'Am', b'Average', b'12', b'10', b'Service', b'Housewife', b'Average', b'Average', b'Govt', b'Asm', b'Large']
Y está recebendo:  b'Good'
X está recebendo:  [b'M', b'MOBC', b'Pass', b'Good', b'Vg', b'Good', b'N', b'Unmarried', b'V', b'Paid', b'Medium', b'Small', b'12', b'Um', b'Business', b'Business', b'Large', b'Poor', b'Govt', b'Asm', b'Average']
Y está recebendo:  b'Average'
X está recebendo:  [b'M', b'G', b'Good', b'Good'

ValueError: could not convert string to float: b'F'